In [1]:
# Importing the necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from keras.initializers import Constant
from keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
# Importing the Tweets Data set 
data = pd.read_csv('....\Tweets.csv')
# Keeping only the neccessary columns
data

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [3]:
data = data[['text','airline_sentiment']]

In [4]:
data['text'] = data['text'].apply(lambda x: x.lower())
#data['text'] = data['text'].apply((lambda x: re.sub('@\w+','',x)))
#data['text'] = data['text'].apply((lambda x: re.sub('http\S+','',x)))

<ipython-input-4-eb60cc312d1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())


In [5]:
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

<ipython-input-5-5a5269ec28b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [6]:
# Printing the size of the Positive, Neutral and Negatives data points in the Tweets data set
print(data[ data['airline_sentiment'] == 'positive'].size)
print(data[ data['airline_sentiment'] == 'negative'].size)
print(data[ data['airline_sentiment'] == 'neutral'].size)

4726
18356
6198


In [7]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [8]:
data

,text,airline_sentiment
0,virginamerica what dhepburn said,neutral
1,virginamerica plus youve added commercials to ...,positive
2,virginamerica i didnt today must mean i need t...,neutral
3,virginamerica its really aggressive to blast o...,negative
4,virginamerica and its a really big bad thing a...,negative
...,...,...
14635,americanair thank you we got on a different fl...,positive
14636,americanair leaving over 20 minutes late fligh...,negative
14637,americanair please bring american airlines to ...,neutral
14638,americanair you have my money you change my fl...,negative


In [9]:
# Tokeninizing the data and padding them
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [10]:
# Splittting of the data set into training and testing sets
Y = pd.get_dummies(data['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9808, 32) (9808, 3)
(4832, 32) (4832, 3)


In [11]:
# CNN
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dropout
embed_dim = 128
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Conv1D(filters=32, kernel_size=5, activation='relu', padding='same'))

model.add(GlobalMaxPooling1D())
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train,Y_train,epochs = 20,batch_size=256,validation_split = 0.33)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 128)           256000    
_________________________________________________________________
conv1d (Conv1D)              (None, 32, 32)            20512     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 281,123
Trainable params: 281,123
Non-trainable params: 0
__________________________________________________

In [12]:
# A Function to print Confusion Matrix, Classification Report and Accuracy Score
def results(labels, pred):
    print(confusion_matrix(labels,pred))  
    print(classification_report(labels,pred))  
    print(accuracy_score(labels, pred))
ans = model.predict(X_test)
labels = [np.where(r==1)[0][0] for r in Y_test]
ans = np.argmax(ans,axis=1)
accuracy_score(ans,labels)
results(ans,labels)

[[2772  360  152]
 [ 224  525  101]
 [  89   99  510]]
              precision    recall  f1-score   support

           0       0.90      0.84      0.87      3284
           1       0.53      0.62      0.57       850
           2       0.67      0.73      0.70       698

    accuracy                           0.79      4832
   macro avg       0.70      0.73      0.71      4832
weighted avg       0.80      0.79      0.79      4832

0.7878725165562914


In [29]:
# Predicting with CNN
twt = ["weather is fantastic."]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if (np.argmax(sentiment) == 0):
  print("negative")
elif (np.argmax(sentiment) == 1):
  print("neutral")
else:
  print('positive')

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0  130   14 1127]]
1/1 - 0s
positive


In [15]:
#TFIDF Machine Learning (SVM) with Prediction
data_2 = data[['text']]
X = data_2['text']
Y = data['airline_sentiment']

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [17]:
vectorizer = TfidfVectorizer()

In [18]:
from sklearn.svm import LinearSVC

In [19]:
clf = LinearSVC()

In [20]:
from sklearn.pipeline import Pipeline

In [21]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [22]:
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [23]:
predictions = text_clf.predict(X_test)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[848  54  22]
 [107 164  31]
 [ 39  32 167]]
              precision    recall  f1-score   support

    negative       0.85      0.92      0.88       924
     neutral       0.66      0.54      0.59       302
    positive       0.76      0.70      0.73       238

    accuracy                           0.81      1464
   macro avg       0.76      0.72      0.74      1464
weighted avg       0.80      0.81      0.80      1464

0.805327868852459


In [25]:
text_clf.predict(["Happy Valentine's day"])

array(['positive'], dtype=object)

In [52]:
from sklearn.ensemble import RandomForestClassifier

In [53]:
clf = RandomForestClassifier()

In [54]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',RandomForestClassifier())])
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier())])

In [55]:
predictions = text_clf.predict(X_test)

In [56]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[892  26   6]
 [176 107  19]
 [100  26 112]]
              precision    recall  f1-score   support

    negative       0.76      0.97      0.85       924
     neutral       0.67      0.35      0.46       302
    positive       0.82      0.47      0.60       238

    accuracy                           0.76      1464
   macro avg       0.75      0.60      0.64      1464
weighted avg       0.75      0.76      0.73      1464

0.7588797814207651


In [57]:
text_clf.predict(["superb"])

array(['neutral'], dtype=object)

In [58]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
#clf.fit(X_train_tfidf,y_train)
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])

In [59]:
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [60]:
predictions = text_clf.predict(X_test)

In [61]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[923   1   0]
 [267  32   3]
 [204   3  31]]
              precision    recall  f1-score   support

    negative       0.66      1.00      0.80       924
     neutral       0.89      0.11      0.19       302
    positive       0.91      0.13      0.23       238

    accuracy                           0.67      1464
   macro avg       0.82      0.41      0.40      1464
weighted avg       0.75      0.67      0.58      1464

0.6734972677595629


In [62]:
text_clf.predict(["superb"])

array(['negative'], dtype='<U8')